# Agenda: CSV files

1. A little background on CSV
2. `read_csv` and `to_csv`
3. filespec
4. `sep`
5. `header`
6. `index_col`
7. `usecols`
8. `names`
9. `dtype`
10. `engine`
11. `parse_dates`
12. `date_format`
13. `comment`

# What are CSV files?

CSV originally stood for "comma-separated values," but now I've heard it can be called "character-separated values."

The idea is that we have a text-file format that contains our records:

- Every record is one line
- On each line, we have multiple fields
- Fields are separated by commas

On the face of it, that seems reasonable! But there are a few problems:

- Because it's a text format, not a binary format, Pandas needs to figure out what type of values should be in each column
- Sometimes, we (or the authors of the file) want to use a different separator. I'm partial to `'\t'` (tab), because it's unlikely to be used in the data.
- Sometimes, the column names are included, and sometimes they aren't.

You might think that CSV files aren't that complex; once you know the separator and a few other pieces of information, you should be able to read it into Python with `str.split`. 

The problem is that CSV allows us to have commas (or whatever separators we're using) *inside* of a data field.

`read_csv` handles the separator inside of a field just fine.

In [1]:
s = 'ab,cde,fg'

s.split(',')

['ab', 'cde', 'fg']

In [2]:
# we can include , inside of a field if we use "" around it
s = 'ab,"cd,e",fg'
s.split(',')

['ab', '"cd', 'e"', 'fg']

# To read a CSV file, we use `pd.read_csv`

Notice that this is a function defined in the `pd` namespace, not a method we call on a data frame. The point is to create a data frame where there wasn't one.

In [4]:
import pandas as pd
from pandas import Series, DataFrame

help(pd.read_csv)

Help on function read_csv in module pandas.io.parsers.readers:

read_csv(filepath_or_buffer: 'FilePath | ReadCsvBuffer[bytes] | ReadCsvBuffer[str]', *, sep: 'str | None | lib.NoDefault' = <no_default>, delimiter: 'str | None | lib.NoDefault' = None, header: "int | Sequence[int] | None | Literal['infer']" = 'infer', names: 'Sequence[Hashable] | None | lib.NoDefault' = <no_default>, index_col: 'IndexLabel | Literal[False] | None' = None, usecols: 'UsecolsArgType' = None, dtype: 'DtypeArg | None' = None, engine: 'CSVEngine | None' = None, converters: 'Mapping[Hashable, Callable] | None' = None, true_values: 'list | None' = None, false_values: 'list | None' = None, skipinitialspace: 'bool' = False, skiprows: 'list[int] | int | Callable[[Hashable], bool] | None' = None, skipfooter: 'int' = 0, nrows: 'int | None' = None, na_values: 'Hashable | Iterable[Hashable] | Mapping[Hashable, Iterable[Hashable]] | None' = None, keep_default_na: 'bool' = True, na_filter: 'bool' = True, verbose: 'bool | 

# How can I use `read_csv`?

The most usual case is to call `pd.read_csv` with a string, a filename on the local filesystem that is in CSV format.  The file can actually be compressed with zip or a few other compression systems, and Pandas will know what to do.

But you can actually pass two other types of values:

- An open file object ("file-like object"), and if it's readable, then it'll work.
- A string containing a URL. `read_csv` will download the content from that URL, and then feed that into `read_csv`.

In [10]:
df = pd.read_csv('albany,ny.csv')

In [11]:
df.head()

,date_time,"albany,ny_maxtempC","albany,ny_mintempC","albany,ny_totalSnow_cm","albany,ny_sunHour","albany,ny_uvIndex","albany,ny_uvIndex.1","albany,ny_moon_illumination","albany,ny_moonrise","albany,ny_moonset",...,"albany,ny_WindChillC","albany,ny_WindGustKmph","albany,ny_cloudcover","albany,ny_humidity","albany,ny_precipMM","albany,ny_pressure","albany,ny_tempC","albany,ny_visibility","albany,ny_winddirDegree","albany,ny_windspeedKmph"
0,2018-12-11 00:00:00,-2,-8,0.0,8.7,2,0,21,10:31 AM,08:22 PM,...,-5,2,9,67,0.0,1023,-5,10,309,1
1,2018-12-11 03:00:00,-2,-8,0.0,8.7,2,0,21,10:31 AM,08:22 PM,...,-6,4,18,68,0.0,1023,-6,10,173,2
2,2018-12-11 06:00:00,-2,-8,0.0,8.7,2,0,21,10:31 AM,08:22 PM,...,-7,2,14,68,0.0,1023,-8,10,168,1
3,2018-12-11 09:00:00,-2,-8,0.0,8.7,2,2,21,10:31 AM,08:22 PM,...,-3,3,8,55,0.0,1021,-4,10,207,3
4,2018-12-11 12:00:00,-2,-8,0.0,8.7,2,2,21,10:31 AM,08:22 PM,...,-2,4,3,50,0.0,1017,-2,10,189,3


# What do we check when we've loaded a CSV file?

1. Did we get the separator right?
2. Are the column names right? Did we get the right row for the column names?
3. Did Pandas guess the dtypes correctly?

In [12]:
df.dtypes

date_time                       object
albany,ny_maxtempC               int64
albany,ny_mintempC               int64
albany,ny_totalSnow_cm         float64
albany,ny_sunHour              float64
albany,ny_uvIndex                int64
albany,ny_uvIndex.1              int64
albany,ny_moon_illumination      int64
albany,ny_moonrise              object
albany,ny_moonset               object
albany,ny_sunrise               object
albany,ny_sunset                object
albany,ny_DewPointC              int64
albany,ny_FeelsLikeC             int64
albany,ny_HeatIndexC             int64
albany,ny_WindChillC             int64
albany,ny_WindGustKmph           int64
albany,ny_cloudcover             int64
albany,ny_humidity               int64
albany,ny_precipMM             float64
albany,ny_pressure               int64
albany,ny_tempC                  int64
albany,ny_visibility             int64
albany,ny_winddirDegree          int64
albany,ny_windspeedKmph          int64
dtype: object

# Writing CSV files

If I want to write a data frame in memory out to a CSV file, I can use `pd.to_csv`. This also has many options, but not quite as many.


In [13]:
df.to_csv('output.csv')

In [14]:
!head output.csv

,date_time,"albany,ny_maxtempC","albany,ny_mintempC","albany,ny_totalSnow_cm","albany,ny_sunHour","albany,ny_uvIndex","albany,ny_uvIndex.1","albany,ny_moon_illumination","albany,ny_moonrise","albany,ny_moonset","albany,ny_sunrise","albany,ny_sunset","albany,ny_DewPointC","albany,ny_FeelsLikeC","albany,ny_HeatIndexC","albany,ny_WindChillC","albany,ny_WindGustKmph","albany,ny_cloudcover","albany,ny_humidity","albany,ny_precipMM","albany,ny_pressure","albany,ny_tempC","albany,ny_visibility","albany,ny_winddirDegree","albany,ny_windspeedKmph"
0,2018-12-11 00:00:00,-2,-8,0.0,8.7,2,0,21,10:31 AM,08:22 PM,07:15 AM,04:21 PM,-10,-5,-5,-5,2,9,67,0.0,1023,-5,10,309,1
1,2018-12-11 03:00:00,-2,-8,0.0,8.7,2,0,21,10:31 AM,08:22 PM,07:15 AM,04:21 PM,-11,-6,-6,-6,4,18,68,0.0,1023,-6,10,173,2
2,2018-12-11 06:00:00,-2,-8,0.0,8.7,2,0,21,10:31 AM,08:22 PM,07:15 AM,04:21 PM,-12,-7,-7,-7,2,14,68,0.0,1023,-8,10,168,1
3,2018-12-11 09:00:00,-2,-8,0.0,8.7,2,2,21,10:31 AM,08:22 PM,07:15 AM,04:21 PM,-11,-3,-3,-3,3

# Exercise: Reading a CSV file

1. In the directory, alongside the Jupyter notebook for this class, you'll find the weather info for Albany, NY. Download it, and create a data frame with it.
2. Write out the data frame to disk, but using `'\t'` as the separator, rather than the default of `,`.

In [15]:
filename = 'albany,ny.csv'

df = pd.read_csv(filename)

In [16]:
df.head()

,date_time,"albany,ny_maxtempC","albany,ny_mintempC","albany,ny_totalSnow_cm","albany,ny_sunHour","albany,ny_uvIndex","albany,ny_uvIndex.1","albany,ny_moon_illumination","albany,ny_moonrise","albany,ny_moonset",...,"albany,ny_WindChillC","albany,ny_WindGustKmph","albany,ny_cloudcover","albany,ny_humidity","albany,ny_precipMM","albany,ny_pressure","albany,ny_tempC","albany,ny_visibility","albany,ny_winddirDegree","albany,ny_windspeedKmph"
0,2018-12-11 00:00:00,-2,-8,0.0,8.7,2,0,21,10:31 AM,08:22 PM,...,-5,2,9,67,0.0,1023,-5,10,309,1
1,2018-12-11 03:00:00,-2,-8,0.0,8.7,2,0,21,10:31 AM,08:22 PM,...,-6,4,18,68,0.0,1023,-6,10,173,2
2,2018-12-11 06:00:00,-2,-8,0.0,8.7,2,0,21,10:31 AM,08:22 PM,...,-7,2,14,68,0.0,1023,-8,10,168,1
3,2018-12-11 09:00:00,-2,-8,0.0,8.7,2,2,21,10:31 AM,08:22 PM,...,-3,3,8,55,0.0,1021,-4,10,207,3
4,2018-12-11 12:00:00,-2,-8,0.0,8.7,2,2,21,10:31 AM,08:22 PM,...,-2,4,3,50,0.0,1017,-2,10,189,3


In [17]:
df.dtypes

date_time                       object
albany,ny_maxtempC               int64
albany,ny_mintempC               int64
albany,ny_totalSnow_cm         float64
albany,ny_sunHour              float64
albany,ny_uvIndex                int64
albany,ny_uvIndex.1              int64
albany,ny_moon_illumination      int64
albany,ny_moonrise              object
albany,ny_moonset               object
albany,ny_sunrise               object
albany,ny_sunset                object
albany,ny_DewPointC              int64
albany,ny_FeelsLikeC             int64
albany,ny_HeatIndexC             int64
albany,ny_WindChillC             int64
albany,ny_WindGustKmph           int64
albany,ny_cloudcover             int64
albany,ny_humidity               int64
albany,ny_precipMM             float64
albany,ny_pressure               int64
albany,ny_tempC                  int64
albany,ny_visibility             int64
albany,ny_winddirDegree          int64
albany,ny_windspeedKmph          int64
dtype: object

In [18]:
df.to_csv('albany.csv', sep='\t')

In [19]:
!head albany.csv

	date_time	albany,ny_maxtempC	albany,ny_mintempC	albany,ny_totalSnow_cm	albany,ny_sunHour	albany,ny_uvIndex	albany,ny_uvIndex.1	albany,ny_moon_illumination	albany,ny_moonrise	albany,ny_moonset	albany,ny_sunrise	albany,ny_sunset	albany,ny_DewPointC	albany,ny_FeelsLikeC	albany,ny_HeatIndexC	albany,ny_WindChillC	albany,ny_WindGustKmph	albany,ny_cloudcover	albany,ny_humidity	albany,ny_precipMM	albany,ny_pressure	albany,ny_tempC	albany,ny_visibility	albany,ny_winddirDegree	albany,ny_windspeedKmph
0	2018-12-11 00:00:00	-2	-8	0.0	8.7	2	0	21	10:31 AM	08:22 PM	07:15 AM	04:21 PM	-10	-5	-5	-5	2	9	67	0.0	1023	-5	10	309	1
1	2018-12-11 03:00:00	-2	-8	0.0	8.7	2	0	21	10:31 AM	08:22 PM	07:15 AM	04:21 PM	-11	-6	-6	-6	4	18	68	0.0	1023	-6	10	173	2
2	2018-12-11 06:00:00	-2	-8	0.0	8.7	2	0	21	10:31 AM	08:22 PM	07:15 AM	04:21 PM	-12	-7	-7	-7	2	14	68	0.0	1023	-8	10	168	1
3	2018-12-11 09:00:00	-2	-8	0.0	8.7	2	2	21	10:31 AM	08:22 PM	07:15 AM	04:21 PM	-11	-3	-3	-3	3	8	55	0.0	1021	-4	10	207	3
4	2018-12-11 12:00:00

In [20]:
df.to_csv('albany.csv', sep='e')

In [21]:
!head albany.csv

e"date_time"e"albany,ny_maxtempC"e"albany,ny_mintempC"ealbany,ny_totalSnow_cmealbany,ny_sunHoure"albany,ny_uvIndex"e"albany,ny_uvIndex.1"ealbany,ny_moon_illuminatione"albany,ny_moonrise"e"albany,ny_moonset"e"albany,ny_sunrise"e"albany,ny_sunset"e"albany,ny_DewPointC"e"albany,ny_FeelsLikeC"e"albany,ny_HeatIndexC"ealbany,ny_WindChillCealbany,ny_WindGustKmphe"albany,ny_cloudcover"ealbany,ny_humiditye"albany,ny_precipMM"e"albany,ny_pressure"e"albany,ny_tempC"ealbany,ny_visibilitye"albany,ny_winddirDegree"e"albany,ny_windspeedKmph"
0e2018-12-11 00:00:00e-2e-8e0.0e8.7e2e0e21e10:31 AMe08:22 PMe07:15 AMe04:21 PMe-10e-5e-5e-5e2e9e67e0.0e1023e-5e10e309e1
1e2018-12-11 03:00:00e-2e-8e0.0e8.7e2e0e21e10:31 AMe08:22 PMe07:15 AMe04:21 PMe-11e-6e-6e-6e4e18e68e0.0e1023e-6e10e173e2
2e2018-12-11 06:00:00e-2e-8e0.0e8.7e2e0e21e10:31 AMe08:22 PMe07:15 AMe04:21 PMe-12e-7e-7e-7e2e14e68e0.0e1023e-8e10e168e1
3e2018-12-11 09:00:00e-2e-8e0.0e8.7e2e2e21e10:31 AMe08:22 PMe07:15 AMe04:21 PMe-11e-3e-3e-3e3e8e55e0.0e10

In [22]:
!head $filename

This is a file
It has information





date_time,"albany,ny_maxtempC","albany,ny_mintempC","albany,ny_totalSnow_cm","albany,ny_sunHour","albany,ny_uvIndex","albany,ny_uvIndex","albany,ny_moon_illumination","albany,ny_moonrise","albany,ny_moonset","albany,ny_sunrise","albany,ny_sunset","albany,ny_DewPointC","albany,ny_FeelsLikeC","albany,ny_HeatIndexC","albany,ny_WindChillC","albany,ny_WindGustKmph","albany,ny_cloudcover","albany,ny_humidity","albany,ny_precipMM","albany,ny_pressure","albany,ny_tempC","albany,ny_visibility","albany,ny_winddirDegree","albany,ny_windspeedKmph"
2018-12-11 00:00:00,-2,-8,0.0,8.7,2,0,21,10:31 AM,08:22 PM,07:15 AM,04:21 PM,-10,-5,-5,-5,2,9,67,0.0,1023,-5,10,309,1
2018-12-11 03:00:00,-2,-8,0.0,8.7,2,0,21,10:31 AM,08:22 PM,07:15 AM,04:21 PM,-11,-6,-6,-6,4,18,68,0.0,1023,-6,10,173,2


In [23]:
# what will happen in this case, if we try to read the CSV file?

df = pd.read_csv(filename)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 8, saw 25


In [36]:
# we can ignore the first few lines of a file, telling Pandas that the headers
# are actually on another line, with the header= keyword argument.

# - Not passing it means that the first line of the file contains the headers
# - Passing it with a value of None means that there are no headers in the file
# - Passing it with an integer value indicates where the headers are

df = pd.read_csv(filename, header=2)  # this means: ignore the first 2 lines, and the 3rd contains the header names

In [37]:
df.head()

,date_time,"albany,ny_maxtempC","albany,ny_mintempC","albany,ny_totalSnow_cm","albany,ny_sunHour","albany,ny_uvIndex","albany,ny_uvIndex.1","albany,ny_moon_illumination","albany,ny_moonrise","albany,ny_moonset",...,"albany,ny_WindChillC","albany,ny_WindGustKmph","albany,ny_cloudcover","albany,ny_humidity","albany,ny_precipMM","albany,ny_pressure","albany,ny_tempC","albany,ny_visibility","albany,ny_winddirDegree","albany,ny_windspeedKmph"
0,2018-12-11 00:00:00,-2,-8,0.0,8.7,2,0,21,10:31 AM,08:22 PM,...,-5,2,9,67,0.0,1023,-5,10,309,1
1,2018-12-11 03:00:00,-2,-8,0.0,8.7,2,0,21,10:31 AM,08:22 PM,...,-6,4,18,68,0.0,1023,-6,10,173,2
2,2018-12-11 06:00:00,-2,-8,0.0,8.7,2,0,21,10:31 AM,08:22 PM,...,-7,2,14,68,0.0,1023,-8,10,168,1
3,2018-12-11 09:00:00,-2,-8,0.0,8.7,2,2,21,10:31 AM,08:22 PM,...,-3,3,8,55,0.0,1021,-4,10,207,3
4,2018-12-11 12:00:00,-2,-8,0.0,8.7,2,2,21,10:31 AM,08:22 PM,...,-2,4,3,50,0.0,1017,-2,10,189,3


In [33]:
!head "albany,ny.csv"

This is a file
It has information





date_time,"albany,ny_maxtempC","albany,ny_mintempC","albany,ny_totalSnow_cm","albany,ny_sunHour","albany,ny_uvIndex","albany,ny_uvIndex","albany,ny_moon_illumination","albany,ny_moonrise","albany,ny_moonset","albany,ny_sunrise","albany,ny_sunset","albany,ny_DewPointC","albany,ny_FeelsLikeC","albany,ny_HeatIndexC","albany,ny_WindChillC","albany,ny_WindGustKmph","albany,ny_cloudcover","albany,ny_humidity","albany,ny_precipMM","albany,ny_pressure","albany,ny_tempC","albany,ny_visibility","albany,ny_winddirDegree","albany,ny_windspeedKmph"
2018-12-11 00:00:00,-2,-8,0.0,8.7,2,0,21,10:31 AM,08:22 PM,07:15 AM,04:21 PM,-10,-5,-5,-5,2,9,67,0.0,1023,-5,10,309,1
2018-12-11 03:00:00,-2,-8,0.0,8.7,2,0,21,10:31 AM,08:22 PM,07:15 AM,04:21 PM,-11,-6,-6,-6,4,18,68,0.0,1023,-6,10,173,2


# We can choose columns

We usually don't need all of the columns from a file. We can choose which should be brought into a data frame with the `usecols` keyword argument. This can take names or integers (starting with 0) to indicate which columns we want.

In [38]:
df = pd.read_csv('nyc_taxi_2020-07.csv',
                 usecols=['passenger_count', 'trip_distance', 'total_amount'])

In [39]:
df.head()

,passenger_count,trip_distance,total_amount
0,1.0,1.50,9.30
1,1.0,9.50,27.80
2,1.0,5.85,22.30
3,1.0,1.90,14.16
4,1.0,1.25,7.80


In [44]:
df = pd.read_csv('nyc_taxi_2020-07.csv',
                 usecols=[10, 8, 9])

In [45]:
df.head()

,DOLocationID,payment_type,fare_amount
0,75,2.0,8.0
1,216,1.0,26.5
2,88,2.0,18.5
3,232,1.0,8.0
4,17,2.0,6.5


# No headers?

If you're reading a file without any headers, then you can specify the columns you want integers, but then assign names of your choosing. You can do this with the `name` keyword argument. Just pass a list (of the same length) of the columns' names.

In [46]:
df = pd.read_csv('nyc_taxi_2020-07.csv',
                 usecols=[10, 8, 9],
                names=['first', 'second', 'third'])
df.head()

/var/folders/rr/0mnyyv811fs5vyp22gf4fxk00000gn/T/ipykernel_71067/1078312068.py:1: DtypeWarning: Columns (8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('nyc_taxi_2020-07.csv',


,first,second,third
0,DOLocationID,payment_type,fare_amount
1,75,2,8
2,216,1,26.5
3,88,2,18.5
4,232,1,8


In [47]:
!unzip airports.zip

Archive:  airports.zip
   creating: airports/
  inflating: airports/airlines.dat   
  inflating: airports/airports.dat   
  inflating: airports/routes.dat     


In [48]:
!mv airports/*.dat .

In [49]:
!rmdir airports


In [50]:
!rm airports.zip

In [52]:
!head -20 airlines.dat

1,"Private flight",\N,"-","N/A","","","Y"
2,"135 Airways",\N,"","GNL","GENERAL","United States","N"
3,"1Time Airline",\N,"1T","RNX","NEXTIME","South Africa","Y"
4,"2 Sqn No 1 Elementary Flying Training School",\N,"","WYT","","United Kingdom","N"
5,"213 Flight Unit",\N,"","TFU","","Russia","N"
6,"223 Flight Unit State Airline",\N,"","CHD","CHKALOVSK-AVIA","Russia","N"
7,"224th Flight Unit",\N,"","TTF","CARGO UNIT","Russia","N"
8,"247 Jet Ltd",\N,"","TWF","CLOUD RUNNER","United Kingdom","N"
9,"3D Aviation",\N,"","SEC","SECUREX","United States","N"
10,"40-Mile Air",\N,"Q5","MLA","MILE-AIR","United States","Y"
11,"4D Air",\N,"","QRT","QUARTET","Thailand","N"
12,"611897 Alberta Limited",\N,"","THD","DONUT","Canada","N"
13,"Ansett Australia",\N,"AN","AAA","ANSETT","Australia","Y"
14,"Abacus International",\N,"1B","","","Singapore","Y"
15,"Abelag Aviation",\N,"W9","AAB","ABG","Belgium","N"
16,"Army Air Corps",\N,"","AAC","ARMYAIR","United Kingdom","N"
17,"Aero Aviation Centre Ltd.",\N,"","AAD

# Exercise: Airline data

1. Create a data frame with information from `airlines.dat`.
2. We only need four columns: The long, full name, the short (three-letter) name, if it exists, the short (two-letter) name, if it exists, and then the country in which it's based.
3. What 10 countries have the greatest number of airlines?

In [58]:
filename = 'airlines.dat'

df = pd.read_csv(filename,
                header=None,
                usecols=[1, 3, 4, 6],
                names=['name', '2LC', '3LC', 'country'])

df.head(20)

,name,2LC,3LC,country
0,Private flight,-,NaN,NaN
1,135 Airways,NaN,GNL,United States
2,1Time Airline,1T,RNX,South Africa
3,2 Sqn No 1 Elementary Flying Training School,NaN,WYT,United Kingdom
4,213 Flight Unit,NaN,TFU,Russia
5,223 Flight Unit State Airline,NaN,CHD,Russia
6,224th Flight Unit,NaN,TTF,Russia
7,247 Jet Ltd,NaN,TWF,United Kingdom
8,3D Aviation,NaN,SEC,United States
9,40-Mile Air,Q5,MLA,United States


In [59]:
df['country'].value_counts().head(20)

country
United States     1080
Mexico             439
United Kingdom     407
Canada             318
Russia             230
Spain              166
Germany            131
France             119
Australia           93
South Africa        91
Italy               90
Ukraine             89
Nigeria             85
Kazakhstan          79
Sweden              70
China               70
Switzerland         60
Brazil              58
Netherlands         52
Austria             50
Name: count, dtype: int64

In [60]:
!ls *,*

albany,ny.csv  chicago,il.csv	   new+york,ny.csv	 springfield,il.csv
boston,ma.csv  los+angeles,ca.csv  san+francisco,ca.csv  springfield,ma.csv


In [61]:
!head boston,ma.csv

date_time,"boston,ma_maxtempC","boston,ma_mintempC","boston,ma_totalSnow_cm","boston,ma_sunHour","boston,ma_uvIndex","boston,ma_uvIndex","boston,ma_moon_illumination","boston,ma_moonrise","boston,ma_moonset","boston,ma_sunrise","boston,ma_sunset","boston,ma_DewPointC","boston,ma_FeelsLikeC","boston,ma_HeatIndexC","boston,ma_WindChillC","boston,ma_WindGustKmph","boston,ma_cloudcover","boston,ma_humidity","boston,ma_precipMM","boston,ma_pressure","boston,ma_tempC","boston,ma_visibility","boston,ma_winddirDegree","boston,ma_windspeedKmph"
2018-12-11 00:00:00,1,-4,0.0,8.7,2,0,21,10:19 AM,08:12 PM,07:03 AM,04:11 PM,-7,-3,0,-3,10,0,57,0.0,1022,-3,10,339,8
2018-12-11 03:00:00,1,-4,0.0,8.7,2,0,21,10:19 AM,08:12 PM,07:03 AM,04:11 PM,-7,-1,1,-1,7,2,57,0.0,1023,-3,10,319,6
2018-12-11 06:00:00,1,-4,0.0,8.7,2,0,21,10:19 AM,08:12 PM,07:03 AM,04:11 PM,-9,-5,-3,-5,8,4,60,0.0,1023,-4,10,334,7
2018-12-11 09:00:00,1,-4,0.0,8.7,2,2,21,10:19 AM,08:12 PM,07:03 AM,04:11 PM,-9,1,1,1,3,6,49,0.0,1022,-1,10,334,

In [62]:
!head chicago,il.csv

date_time,"chicago,il_maxtempC","chicago,il_mintempC","chicago,il_totalSnow_cm","chicago,il_sunHour","chicago,il_uvIndex","chicago,il_uvIndex","chicago,il_moon_illumination","chicago,il_moonrise","chicago,il_moonset","chicago,il_sunrise","chicago,il_sunset","chicago,il_DewPointC","chicago,il_FeelsLikeC","chicago,il_HeatIndexC","chicago,il_WindChillC","chicago,il_WindGustKmph","chicago,il_cloudcover","chicago,il_humidity","chicago,il_precipMM","chicago,il_pressure","chicago,il_tempC","chicago,il_visibility","chicago,il_winddirDegree","chicago,il_windspeedKmph"
2018-12-11 00:00:00,1,-2,0.0,8.7,2,0,21,10:26 AM,08:22 PM,07:08 AM,04:19 PM,-5,-7,-1,-7,33,0,76,0.0,1021,-1,10,231,22
2018-12-11 03:00:00,1,-2,0.0,8.7,2,0,21,10:26 AM,08:22 PM,07:08 AM,04:19 PM,-4,-6,-1,-6,33,0,80,0.0,1021,-1,10,243,21
2018-12-11 06:00:00,1,-2,0.0,8.7,2,0,21,10:26 AM,08:22 PM,07:08 AM,04:19 PM,-5,-8,-2,-8,28,5,83,0.0,1020,-2,10,240,19
2018-12-11 09:00:00,1,-2,0.0,8.7,2,2,21,10:26 AM,08:22 PM,07:08 AM,04:19 PM,-5,-

In [69]:
df = pd.read_csv('chicago,il.csv',
                 usecols=[0, 1, 2],
                 header=0,
                 names=['date_time', 'maxtemp', 'mintemp'])
df

,date_time,maxtemp,mintemp
0,2018-12-11 00:00:00,1,-2
1,2018-12-11 03:00:00,1,-2
2,2018-12-11 06:00:00,1,-2
3,2018-12-11 09:00:00,1,-2
4,2018-12-11 12:00:00,1,-2
...,...,...,...
723,2019-03-11 09:00:00,3,-2
724,2019-03-11 12:00:00,3,-2
725,2019-03-11 15:00:00,3,-2
726,2019-03-11 18:00:00,3,-2


In [72]:
# if we have many files that fit the same filename pattern,
# and we want to turn them all into data frames, and then
# we want to combine those data frames into a single data frame, we can use a list comprehension

import glob

all_dfs = [pd.read_csv(one_filename,
                       header=0,
                       usecols=[0,1,2],
                       names=['date_time', 'maxtemp', 'mintemp'])
                       .assign(city=one_filename,
                              state=one_filename)
           for one_filename in glob.glob('*,*.csv')]



In [76]:
# let's create a dict, with a dict comprehension!

all_dfs = {one_filename : pd.read_csv(one_filename,
                       header=0,
                       usecols=[0,1,2],
                       names=['date_time', 'maxtemp', 'mintemp'])
                       .assign(city=one_filename,
                              state=one_filename)
           for one_filename in glob.glob('*,*.csv')}



In [77]:
# pd.concat creates one new data frame from all those in its list
pd.concat(all_dfs.values())

,date_time,maxtemp,mintemp,city,state
0,2018-12-11 00:00:00,13,8,"san+francisco,ca.csv","san+francisco,ca.csv"
1,2018-12-11 03:00:00,13,8,"san+francisco,ca.csv","san+francisco,ca.csv"
2,2018-12-11 06:00:00,13,8,"san+francisco,ca.csv","san+francisco,ca.csv"
3,2018-12-11 09:00:00,13,8,"san+francisco,ca.csv","san+francisco,ca.csv"
4,2018-12-11 12:00:00,13,8,"san+francisco,ca.csv","san+francisco,ca.csv"
...,...,...,...,...,...
723,2019-03-11 09:00:00,3,-2,"chicago,il.csv","chicago,il.csv"
724,2019-03-11 12:00:00,3,-2,"chicago,il.csv","chicago,il.csv"
725,2019-03-11 15:00:00,3,-2,"chicago,il.csv","chicago,il.csv"
726,2019-03-11 18:00:00,3,-2,"chicago,il.csv","chicago,il.csv"


# dtype

Every column has a dtype. How is it set? By default, Pandas will examine all of the values in a column, and then decide:

- If it has only digits, then it uses int64
- If it has digits + a decimal point, then it uses float64
- In all other cases, it uses strings (`object`)

This has at least two drawbacks:

1. It takes time for Pandas to examine and decide for a column.
2. Its defaults are terrible for many values.
3. If you have a lot of data, then Pandas will load it in chunks. If you have different values in different rows, then it might get confused, and assign the wrong dtype. If this happens, you'll typically get a warning telling you that it ran out of memory. You can then specify the dtypes or pass `low_memory=False`.

We can avoid these issues by passing a `dtype` keyword argument to `read_csv`. `dtype` takes a dict; the keys are the column names and the values are the dtypes we want to associate with them.

In [79]:
df = pd.read_csv('nyc_taxi_2020-07.csv',
                 usecols=['passenger_count', 'trip_distance', 'total_amount'],
                 dtype={'passenger_count':pd.Int64Dtype(),
                        'trip_distance':'float',
                        'total_amount':'float'})

In [80]:
df.head()

,passenger_count,trip_distance,total_amount
0,1,1.50,9.30
1,1,9.50,27.80
2,1,5.85,22.30
3,1,1.90,14.16
4,1,1.25,7.80


In [81]:
df.dtypes

passenger_count      Int64
trip_distance      float64
total_amount       float64
dtype: object

# Exercise: Specifying dtypes

1. Read `nyc_taxi_2020-07.csv` into a data frame. How long does it take? (You can use `%timeit` to find out.)
2. Now read only four columns into the data frame -- `passenger_count`, `trip_distance`, `total_amount`, `tip_amount`. Does Pandas read this faster now?
3. Specify their dtypes, and time the loading. Is it even faster?

In [83]:
%timeit pd.read_csv('nyc_taxi_2020-07.csv', low_memory=False)

1.45 s ± 20.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [85]:
%%timeit

pd.read_csv('nyc_taxi_2020-07.csv', 
                    usecols=['passenger_count', 'trip_distance',
                             'total_amount', 'tip_amount'],
                    low_memory=False)

665 ms ± 10.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [87]:
%%timeit

pd.read_csv('nyc_taxi_2020-07.csv', 
                    usecols=['passenger_count', 'trip_distance',
                             'total_amount', 'tip_amount'],
                    dtype={'passenger_count':float,
                           'trip_distance': float,
                           'total_amount':float,
                           'tip_amount':float},
                    low_memory=False)

763 ms ± 9.23 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# PyArrow can help us to load faster

Right now, we're (by default) using the Pandas engine to load CSV files. But PyArrow, which is a super-fast in-memory storage system, has its own loader for CSV. We can specify that we want to use it by saying `engine='pyarrow'`.

In [88]:
%timeit pd.read_csv('nyc_taxi_2020-07.csv', engine='pyarrow')

93.6 ms ± 3.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [89]:
%%timeit

pd.read_csv('nyc_taxi_2020-07.csv', 
                    usecols=['passenger_count', 'trip_distance',
                             'total_amount', 'tip_amount'],
                    engine='pyarrow')

13.9 ms ± 96.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [90]:
%%timeit

pd.read_csv('nyc_taxi_2020-07.csv', 
                    usecols=['passenger_count', 'trip_distance',
                             'total_amount', 'tip_amount'],
                    dtype={'passenger_count':float,
                           'trip_distance': float,
                           'total_amount':float,
                           'tip_amount':float},
                    engine='pyarrow')

15.6 ms ± 106 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Dates and times

If we use the builtin Pandas engine for reading CSV files, it will not try to parse datetime information. PyArrow's parser will, however.

If you want to tell Pandas that a column should be interpreted as a datetime, do that by passing a list of strings (column names) to `parse_dates`.

In [92]:
df = pd.read_csv('nyc_taxi_2020-07.csv', 
                    usecols=['tpep_pickup_datetime',
                             'tpep_dropoff_datetime',
                             'passenger_count', 'trip_distance',
                             'total_amount', 'tip_amount'],
                    parse_dates=['tpep_pickup_datetime',
                                 'tpep_dropoff_datetime'],
                    low_memory=False)
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,tip_amount,total_amount
0,2020-07-01 00:25:32,2020-07-01 00:33:39,1.0,1.50,0.00,9.30
1,2020-07-01 00:03:19,2020-07-01 00:25:43,1.0,9.50,0.00,27.80
2,2020-07-01 00:15:11,2020-07-01 00:29:24,1.0,5.85,0.00,22.30
3,2020-07-01 00:30:49,2020-07-01 00:38:26,1.0,1.90,2.36,14.16
4,2020-07-01 00:31:26,2020-07-01 00:38:02,1.0,1.25,0.00,7.80


In [93]:
df.dtypes

tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
tip_amount                      float64
total_amount                    float64
dtype: object

# Ways to affect the parsing of dates

1. `parse_dates`, pass it a list of column names (or numbers) that should be parsed as dates
2. `dayfirst`, which defaults to `False`, indicates whether XX-YY-ZZZZ dates should be interpreted as American dates (with the month first) or European dates (with the day first).
3. `date_format`, where we can specify what the datetime format should look like.

In [95]:
!head -20 albany,ny.csv

This is a file
It has information





date_time,"albany,ny_maxtempC","albany,ny_mintempC","albany,ny_totalSnow_cm","albany,ny_sunHour","albany,ny_uvIndex","albany,ny_uvIndex","albany,ny_moon_illumination","albany,ny_moonrise","albany,ny_moonset","albany,ny_sunrise","albany,ny_sunset","albany,ny_DewPointC","albany,ny_FeelsLikeC","albany,ny_HeatIndexC","albany,ny_WindChillC","albany,ny_WindGustKmph","albany,ny_cloudcover","albany,ny_humidity","albany,ny_precipMM","albany,ny_pressure","albany,ny_tempC","albany,ny_visibility","albany,ny_winddirDegree","albany,ny_windspeedKmph"
2018-12-11 00:00:00,-2,-8,0.0,8.7,2,0,21,10:31 AM,08:22 PM,07:15 AM,04:21 PM,-10,-5,-5,-5,2,9,67,0.0,1023,-5,10,309,1
2018-12-11 03:00:00,-2,-8,0.0,8.7,2,0,21,10:31 AM,08:22 PM,07:15 AM,04:21 PM,-11,-6,-6,-6,4,18,68,0.0,1023,-6,10,173,2
2018-12-11 06:00:00,-2,-8,0.0,8.7,2,0,21,10:31 AM,08:22 PM,07:15 AM,04:21 PM,-12,-7,-7,-7,2,14,68,0.0,1023,-8,10,168,1
2018-12-11 09:00:00,-2,-8,0.0,8.7,2,2,21,10:31 AM,08:22 PM,07:1

# Exercise: Albany temps and snowfall

1. Read the Albany, NY data into a data frame. Make sure that `date_time` is treated as a `datetime` value.
2. You can extract the month from `datetime` with the `.dt.month` method. Use that to retrieve the month, and then calculate the average snowfall in Albany for each month.

In [102]:
filename = 'albany,ny.csv'

df = pd.read_csv(filename, header=2)
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728 entries, 0 to 727
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   date_time                    728 non-null    object 
 1   albany,ny_maxtempC           728 non-null    int64  
 2   albany,ny_mintempC           728 non-null    int64  
 3   albany,ny_totalSnow_cm       728 non-null    float64
 4   albany,ny_sunHour            728 non-null    float64
 5   albany,ny_uvIndex            728 non-null    int64  
 6   albany,ny_uvIndex.1          728 non-null    int64  
 7   albany,ny_moon_illumination  728 non-null    int64  
 8   albany,ny_moonrise           728 non-null    object 
 9   albany,ny_moonset            728 non-null    object 
 10  albany,ny_sunrise            728 non-null    object 
 11  albany,ny_sunset             728 non-null    object 
 12  albany,ny_DewPointC          728 non-null    int64  
 13  albany,ny_FeelsLikeC

In [104]:
filename = 'albany,ny.csv'

df = pd.read_csv(filename, header=2, parse_dates=['date_time'])
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728 entries, 0 to 727
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date_time                    728 non-null    datetime64[ns]
 1   albany,ny_maxtempC           728 non-null    int64         
 2   albany,ny_mintempC           728 non-null    int64         
 3   albany,ny_totalSnow_cm       728 non-null    float64       
 4   albany,ny_sunHour            728 non-null    float64       
 5   albany,ny_uvIndex            728 non-null    int64         
 6   albany,ny_uvIndex.1          728 non-null    int64         
 7   albany,ny_moon_illumination  728 non-null    int64         
 8   albany,ny_moonrise           728 non-null    object        
 9   albany,ny_moonset            728 non-null    object        
 10  albany,ny_sunrise            728 non-null    object        
 11  albany,ny_sunset             728 non-null    

In [105]:
df.groupby(df['date_time'].dt.month)['albany,ny_totalSnow_cm'].mean()

date_time
1     2.341935
2     1.296429
3     0.890909
12    0.090476
Name: albany,ny_totalSnow_cm, dtype: float64

# Non-CSV CSV files

You can treat a file as a CSV file if it has (a) regular separators and (b) can be handled with the various options.

In [106]:
!head -20 /etc/passwd

##
# User Database
# 
# Note that this file is consulted directly only when the system is running
# in single-user mode.  At other times this information is provided by
# Open Directory.
#
# See the opendirectoryd(8) man page for additional information about
# Open Directory.
##
nobody:*:-2:-2:Unprivileged User:/var/empty:/usr/bin/false
root:*:0:0:System Administrator:/var/root:/bin/sh
daemon:*:1:1:System Services:/var/root:/usr/bin/false
_uucp:*:4:4:Unix to Unix Copy Protocol:/var/spool/uucp:/usr/sbin/uucico
_taskgated:*:13:13:Task Gate Daemon:/var/empty:/usr/bin/false
_networkd:*:24:24:Network Services:/var/networkd:/usr/bin/false
_installassistant:*:25:25:Install Assistant:/var/empty:/usr/bin/false
_lp:*:26:26:Printing Services:/var/spool/cups:/usr/bin/false
_postfix:*:27:27:Postfix Mail Server:/var/spool/postfix:/usr/bin/false
_scsd:*:31:31:Service Configuration Service:/var/empty:/usr/bin/false


In [110]:
pd.read_csv('/etc/passwd',
            sep=':',
           comment='#',
           header=None,
           names=['username', 'pw', 'uid', 'gid', 'name', 'homedir', 'shell'])

,username,pw,uid,gid,name,homedir,shell
0,nobody,*,-2,-2,Unprivileged User,/var/empty,/usr/bin/false
1,root,*,0,0,System Administrator,/var/root,/bin/sh
2,daemon,*,1,1,System Services,/var/root,/usr/bin/false
3,_uucp,*,4,4,Unix to Unix Copy Protocol,/var/spool/uucp,/usr/sbin/uucico
4,_taskgated,*,13,13,Task Gate Daemon,/var/empty,/usr/bin/false
...,...,...,...,...,...,...,...
117,_audiomxd,*,294,294,Audio and MediaExperience Daemon,/var/db/audiomxd,/usr/bin/false
118,_terminusd,*,295,295,Terminus,/var/empty,/usr/bin/false
119,_neuralengine,*,296,296,AppleNeuralEngine,/var/db/neuralengine,/usr/bin/false
120,_eligibilityd,*,297,297,OS Eligibility Daemon,/var/db/eligibilityd,/usr/bin/false
